In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import telegram_send

ModuleNotFoundError: No module named 'telegram_send'

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
urls = ['https://etherscan.io/txs?a=0x8b3f4eb783270aefaac9238ac1d165a433c8fbf3&f=2',
       'https://etherscan.io/txs?a=0xb8659f9034fdb8608598f7295b6b82869c379510&f=2',
       'https://etherscan.io/txs?a=0x820880c9040041d6a514e069ec0a2f37997325d4&f=2',
       'https://etherscan.io/txs?a=0xbdd95abe8a7694ccd77143376b0fbea183e6a740&f=2',
       'https://etherscan.io/txs?a=0xfdd50de023c9a705d9086bf821d15c7450ee93bf&f=2']



In [ ]:
def etherscan_address_scraper():
    dfs=[]
    for url in urls:
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.content, 'lxml')
        table = soup.find_all('table')[0]
        df_list = pd.read_html(str(table))
        df = pd.concat(df_list)  # this line is what you're missing
        dfs.append(df)

    final_df = pd.concat(dfs)

    final_df.drop(['Unnamed: 0', 'Block'], axis=1, inplace=True)
    final_df.rename(columns={"Method": "Nature", "Unnamed: 4": "Time", "Unnamed: 6" : "In/Out","Unnamed: 9": "Gas fees"}, inplace=True )

    final_df["Nature"] = final_df["Nature"].str.lower()
    final_df['Time'] = final_df['Time'].str.replace(r'\ ago', '')

    final_df['Hours'] = final_df['Time'].str[-15:-10]
    final_df['Mins'] = final_df['Time'].str[-7:-4]
    final_df['Hours'] = final_df['Hours'].str.extract('(\d+)', expand=False)
    final_df['Mins'] = final_df['Mins'].str.extract('(\d+)', expand=False)
    final_df['Hours'] = final_df['Hours'].fillna(0)
    final_df["Hours"] = pd.to_numeric(final_df["Hours"], downcast="float")
    final_df["Mins"] = pd.to_numeric(final_df["Mins"], downcast="float")

    conditions = [
        (final_df['Hours'] == 0) & (final_df['Mins']<=1),
        (final_df['Nature'].str.contains("mint").any())]

    last_transaction = final_df.iloc[0,[3,2,5,6]]

    message = last_transaction.to_string()

    if conditions is True:
        telegram_send.send(messages=[message])

    else:
        print(message)

while True:
    etherscan_address_scraper()
    time.sleep(30)